In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sber_data = pd.read_csv('data/sber_data.csv')

### ОБНАРУЖЕНИЕ И ЛИКВИДАЦИЯ ДУБЛИКАТОВ

✍️ Проверим, есть у нас записи с одинаковым id: для этого сравним число уникальных значений в столбце id с числом строк. Число уникальных значений вычислим с помощью метода nunique():

In [2]:
sber_data['id'].nunique() == sber_data.shape[0]

True

###### Вроде бы всё в порядке: каждой записи в таблице соответствует свой уникальный идентификатор. Но это ещё не означает, что в таблице нет дубликатов!

Чтобы отследить дубликаты, можно воспользоваться методом <b>duplicated()</b>, который возвращает булеву маску для фильтрации.

✍️ Найдём число полных дубликатов таблице sber_data. Предварительно создадим список столбцов dupl_columns, по которым будем искать совпадения (все столбцы, не включая id). 

In [3]:
# Создадим список столбцов dupl_columns, по которым будем искать совпадения (все, кроме ID)
dupl_columns = list(sber_data.columns)
dupl_columns.remove('id')

# Создадим маску дубликатов
mask = sber_data.duplicated(subset=dupl_columns)

# Произведём фильтрацию
sber_duplicates = sber_data[mask]

# Выведем число строк в результирующем DataFrame
print(f'Число найденных дубликатов: {sber_duplicates.shape[0]}')

Число найденных дубликатов: 562


Теперь нам необходимо от них избавиться. Для этого легче всего воспользоваться методом <b>drop_duplicates()</b>, который удаляет повторяющиеся записи из таблицы. 

✍️ Создадим новую таблицу sber_dedupped, которая будет версией исходной таблицы, очищенной от полных дубликатов.

In [4]:
sber_dedupped = sber_data.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {sber_dedupped.shape[0]}')

Результирующее число записей: 29909


### НЕИНФОРМАТИВНЫЕ ПРИЗНАКИ

Чтобы считать признак неинформативным, прежде всего нужно задать какой-то определённый порог. Например, часто используют пороги в 0.95 и 0.99. Это означает: признак неинформативен, если в нем 95 % (99 %) одинаковых значений или же 95 % (99 %) данных полностью уникальны.

<b>Разберём алгоритм:</b>

→ Создаём пустой список <b>low_information_cols</b>, куда будем добавлять названия признаков, которые мы посчитаем неинформативными.

→ В <b>цикле</b> пройдёмся по всем именам столбцов в таблице и для каждого будем совершать следующие действия:

* рассчитаем <b>top_freq</b> — наибольшую относительную частоту с помощью метода value_counts() с параметром normalize=True. Метод вернёт долю от общих данных, которую занимает каждое уникальное значение в признаке.

Например, для столбца oil_chemistry_raion (нефтехимический район) результат будет следующим:<br>
<i>no 0.990286<br>
yes 0.009714</i>

Отсюда нам нужен максимум.

* рассчитаем <b>nunique_ratio</b> — отношение числа уникальных значений в столбце к размеру всего столбца. Число уникальных значений в столбце получим с помощью метода nunique(), а размер признака — с помощью метода count(). Например, для столбца id число уникальных значений — 30471; оно же равно размеру таблицы. Поэтому результат отношения будет 1.
* сравним каждое из полученных чисел с пороговым значением (у нас это 0.95) и добавим в список неинформативных признаков, если условие истинно.

In [5]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in sber_data.columns:
    #наибольшая относительная частота в признаке
    top_freq = sber_data[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = sber_data[col].nunique() / sber_data[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')

id: 100.0% уникальных значений
oil_chemistry_raion: 99.03% одинаковых значений
railroad_terminal_raion: 96.27% одинаковых значений
nuclear_reactor_raion: 97.17% одинаковых значений
big_road1_1line: 97.44% одинаковых значений
mosque_count_1000: 98.08% одинаковых значений


Итак, мы нашли шесть неинформативных признаков. Теперь можно удалить их с помощью метода drop(), передав результирующий список в его аргументы.

In [6]:
information_sber_data = sber_data.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {information_sber_data.shape[1]}')

Результирующее число признаков: 55


### ВАЖНОСТЬ ПРИЗНАКОВ

<b>Важность признака</b> - это степень влияния на целевой признак (тот, который мы хотим предсказать).

Признаки, которые обладают низкой важностью, называют <b>нерелевантными</b> признаками. 